In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import math
import tensorflow as tf
from tensorflow.python import keras
from keras.layers import Input, Dense, SimpleRNN, RNN, LSTM
from keras.models import Model
from keras.models import Sequential
from sklearn import preprocessing
from pathlib import Path

In [2]:
# 讀取資料
csv_files = Path('/Users/nuo/Documents/DA/kaggle/data-analytics-datagame').glob('*.csv')
data = {}
for file in csv_files:
    data[os.path.splitext(file.name)[0]] = pd.read_csv(file)
    print(file)

/Users/nuo/Documents/DA/kaggle/data-analytics-datagame/df_event_y_new.csv
/Users/nuo/Documents/DA/kaggle/data-analytics-datagame/reduce_new.csv
/Users/nuo/Documents/DA/kaggle/data-analytics-datagame/train_source_events.csv
/Users/nuo/Documents/DA/kaggle/data-analytics-datagame/test_source_events.csv
/Users/nuo/Documents/DA/kaggle/data-analytics-datagame/sample.csv
/Users/nuo/Documents/DA/kaggle/data-analytics-datagame/df_merge.csv
/Users/nuo/Documents/DA/kaggle/data-analytics-datagame/train_target_events.csv
/Users/nuo/Documents/DA/kaggle/data-analytics-datagame/light_train_target_labels.csv
/Users/nuo/Documents/DA/kaggle/data-analytics-datagame/light_train_source_labels.csv
/Users/nuo/Documents/DA/kaggle/data-analytics-datagame/df_event_test_reduce.csv
/Users/nuo/Documents/DA/kaggle/data-analytics-datagame/light_test_source_labels.csv


In [3]:
#set index    
df_light = data['light_train_source_labels'].set_index('user_id', drop=True)
df_light_y = data['light_train_target_labels'].set_index('user_id', drop=True)
df_light_test = data['light_test_source_labels'].set_index('user_id', drop=True)
sample = data['sample']
df_event = data['train_source_events']
df_event_y = data['train_target_events']
df_event_test = data['test_source_events']
df_reduce = data['reduce_new']
df_event_y_reduce = data['df_event_y_new']
df_event_test_reduce = data['df_event_test_reduce']

In [4]:
#刪除不必要欄位資料、標準化
df_reduce = df_reduce.drop(['Unnamed: 0', 'Unnamed: 0.1', 'time_slot_2', 'event_time', 'until', 'device_id', 'session_id'], axis=1)
df_reduce_group = df_reduce.groupby(['user_id', 'time_slot_1']).mean().reset_index()
scaler = preprocessing.StandardScaler().fit(df_reduce_group)
df_reduce_group_scaled = scaler.transform(df_reduce_group)

In [5]:
#刪除不必要欄位資料、標準化
df_event_test_reduce = df_event_test_reduce.drop(['Unnamed: 0', 'time_slot_2', 'event_time', 'until', 'device_id', 'session_id'], axis=1)
df_event_test_reduce_group = df_event_test_reduce.groupby(['user_id', 'time_slot_1']).mean().reset_index()
scaler_test = preprocessing.StandardScaler().fit(df_event_test_reduce_group)
df_event_test_reduce_group_scaled = scaler_test.transform(df_event_test_reduce_group)

In [6]:
#合併資料
user = 0
count = 0
time = 0
X_df = []
while user < 30460 or count < 172394:
    if user == 30460 or count == 172394:
        break
    if df_reduce_group['user_id'][count] == user:
        if time == df_reduce_group['time_slot_1'][count]:
            X_df.append(df_reduce_group_scaled[count])
            count += 1
            time += 1
        else:
            X_df.append([0 for zero in range(8)])
            time += 1
    elif time < 28:
        for i in range(28-time):
            X_df.append([0 for zero in range(8)])
            time = 28
    if time == 28:
        time = 0
        user += 1

In [7]:
for i in range(28-time):
    X_df.append([0 for zero in range(8)])

In [8]:
X_df

[array([-1.73106141, -1.5597876 , -0.53876998, -0.26035274, -0.45569276,
        -1.03236154, -0.34667815, -0.70645212]),
 array([-1.73106141, -1.43795596, -0.50532367,  0.6196829 , -0.28057646,
        -1.03236154, -0.3885877 , -0.70645212]),
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 array([-1.73106141, -1.07246104,  0.08225198, -0.27439417, -0.30003383,
        -1.03236154, -0.59813544, -0.70645212]),
 array([-1.73106141, -0.9506294 , -0.46581054,  0.6707181 , -0.28057646,
        -1.03236154, -1.10105002, -0.70645212]),
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 array([-1.73106141, -0.46330284, -0.50911872, -0.17563822, -0.17031804,
        -1.03236154, -0.3885877 , -0.70645212]),
 [0, 0, 0, 0, 0, 0, 0, 0],
 array([-1.73106141, -0.21963956,  0.15032645, -0.47230848, -0.28057646,
        -1.03236154, -1.10105002, -0.70645212]),
 [0, 0, 0, 0, 0, 0, 0, 0],
 array([-1.73106141,  0.02402372, -0.17999716, -0.30779939, -0.30602071,
      

In [9]:
print(user, count, time)

30459 172394 22


In [10]:
X_df_reshape = pd.DataFrame(X_df)
#X_df_reshape = X_df_reshape.values.reshape(30460, 8, 28)

In [11]:
X_df_reshape

#輸出資料
X_df_reshape.to_csv('/Users/nuo/desktop/X_df_reshape.csv')

In [12]:
len(df_event_test_reduce_group_scaled)

42715

In [13]:
#依照時間區分資料
X_train_total = [[] for i in range(28)]
for i in range(len(X_df_reshape)):
    a = i % 28
    X_train_total[a].append(X_df_reshape.loc[i])

In [14]:
#合併資料
user_test = 30460
count_test = 0
time_test = 0
X_df_test = []
while user_test < 38076 or count_test < 42715:
    if user_test == 38076 or count_test == 42715:
        break
    if df_event_test_reduce_group['user_id'][count_test] == user_test:
        if time_test == df_event_test_reduce_group['time_slot_1'][count_test]:
            X_df_test.append(df_event_test_reduce_group_scaled[count_test])
            count_test += 1
            time_test += 1
        else:
            X_df_test.append([0 for zero in range(8)])
            time_test += 1
    elif time_test < 28:
        for i in range(28-time_test):
            X_df_test.append([0 for zero in range(8)])
            time_test = 28
    if time_test == 28:
        time_test = 0
        user_test += 1

In [15]:
print(len(X_df_test))
print(user_test, count_test, time_test)

213242
38075 42715 22


In [16]:
for i in range(28-time_test):
    X_df_test.append([0 for zero in range(8)])
    
print(len(X_df_test))

213248


In [17]:
X_df_test_reshape = pd.DataFrame(X_df_test)
#X_df_test_reshape = X_df_test_reshape.values.reshape(7616, 8, 28)

In [18]:
print(X_df_reshape.shape)
print(X_df_reshape)

(852880, 8)
               0         1         2         3         4         5         6  \
0      -1.731061 -1.559788 -0.538770 -0.260353 -0.455693 -1.032362 -0.346678   
1      -1.731061 -1.437956 -0.505324  0.619683 -0.280576 -1.032362 -0.388588   
2       0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
3       0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
4      -1.731061 -1.072461  0.082252 -0.274394 -0.300034 -1.032362 -0.598135   
...          ...       ...       ...       ...       ...       ...       ...   
852875  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
852876  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
852877  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
852878  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
852879  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

               7  
0      -

In [19]:
#依照時間區分資料
X_test_total = [[] for i in range(28)]
for i in range(len(X_df_test_reshape)):
    a = i % 28
    X_test_total[a].append(X_df_test_reshape.loc[i])
#X_test_total

In [20]:
df_light_y = df_light_y.reset_index(drop=True)
df_light_y = df_light_y.astype(int)
df_light_y.columns[0]

'time_slot_0'

X_train = X_df_reshape
y_train = df_light_y
X_test = X_df_test_reshape

#2D to 1D
nsamples, nx, ny = X_train.shape
X_train = X_train.reshape((nsamples,nx*ny))

nsamples, nx, ny = X_test.shape
X_test = X_test.reshape((nsamples,nx*ny))

X_train = np.reshape(np.array(X_train), (X_train.shape[0], X_train.shape[1], 1))
X_train_total

#test_dane_rnn = np.reshape(np.array(test_dane), (test_dane.shape[0], test_dane.shape[1], 1))

In [24]:
from sklearn.naive_bayes import BernoulliNB

for t in range(28):
    X_train = pd.DataFrame(X_train_total[t])
    y_train = pd.DataFrame(df_light_y[df_light_y.columns[t]])
    X_test = pd.DataFrame(X_test_total[t])
    X_train = np.reshape(X_train, (X_train.shape[0], 8))
    X_test = np.reshape(X_test, (X_test.shape[0], 8))
    model = BernoulliNB().fit(X_train, y_train) 
    predictions = model.predict(X_test)

    sample[sample.columns[t + 1]] = predictions

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/uti

In [25]:
#GaussianNB + t
from sklearn.naive_bayes import GaussianNB

for t in range(28):
    X_train = pd.DataFrame(X_train_total[t])
    y_train = pd.DataFrame(df_light_y[df_light_y.columns[t]])
    X_test = pd.DataFrame(X_test_total[t])
    X_train = np.reshape(X_train, (X_train.shape[0], 8))
    X_test = np.reshape(X_test, (X_test.shape[0], 8))
    model = GaussianNB().fit(X_train, y_train) # C=1, kernel=rbf, gamma=auto
    predictions = model.predict(X_test)

    sample[sample.columns[t + 1]] = predictions

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/uti

In [26]:
# XGBoost 
from xgboost import XGBClassifier

for t in range(28):
    X_train = pd.DataFrame(X_train_total[t])
    y_train = pd.DataFrame(df_light_y[df_light_y.columns[t]])
    X_test = pd.DataFrame(X_test_total[t])
    X_train = np.reshape(X_train, (X_train.shape[0], 8))
    X_test = np.reshape(X_test, (X_test.shape[0], 8))
    model = XGBClassifier().fit(X_train, y_train) # C=1, kernel=rbf, gamma=auto
    predictions = model.predict(X_test)

    sample[sample.columns[t + 1]] = predictions

In [27]:
#DecisionTree
from sklearn import tree

for t in range(28):
    X_train = pd.DataFrame(X_train_total[t])
    y_train = pd.DataFrame(df_light_y[df_light_y.columns[t]])
    X_test = pd.DataFrame(X_test_total[t])
    X_train = np.reshape(X_train, (X_train.shape[0], 8))
    X_test = np.reshape(X_test, (X_test.shape[0], 8))
    model = tree.DecisionTreeClassifier().fit(X_train, y_train) # C=1, kernel=rbf, gamma=auto
    predictions = model.predict(X_test)

    sample[sample.columns[t + 1]] = predictions

In [28]:
#RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

for t in range(28):
    X_train = pd.DataFrame(X_train_total[t])
    y_train = pd.DataFrame(df_light_y[df_light_y.columns[t]])
    X_test = pd.DataFrame(X_test_total[t])
    X_train = np.reshape(X_train, (X_train.shape[0], 8))
    X_test = np.reshape(X_test, (X_test.shape[0], 8))
    model = RandomForestClassifier(n_estimators = 2).fit(X_train, y_train)
    predictions = model.predict(X_test)

    sample[sample.columns[t + 1]] = predictions

/var/folders/23/ylgmhvxn0pb2wzzl35qy5c3m0000gn/T/ipykernel_29660/650904556.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model = RandomForestClassifier(n_estimators = 2).fit(X_train, y_train)
/var/folders/23/ylgmhvxn0pb2wzzl35qy5c3m0000gn/T/ipykernel_29660/650904556.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model = RandomForestClassifier(n_estimators = 2).fit(X_train, y_train)
/var/folders/23/ylgmhvxn0pb2wzzl35qy5c3m0000gn/T/ipykernel_29660/650904556.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model = RandomForestClassifier(n_estimators = 2).fit(X_train, y_train)
/var/folders/23/ylgmhvxn0pb2wzzl35qy5c3m0000gn/T/ipykernel_29660/65090

In [29]:
#svm
from sklearn.svm import SVC

for t in range(28):
    X_train = pd.DataFrame(X_train_total[t])
    y_train = pd.DataFrame(df_light_y[df_light_y.columns[t]])
    X_test = pd.DataFrame(X_test_total[t])
    X_train = np.reshape(X_train, (X_train.shape[0], 8))
    X_test = np.reshape(X_test, (X_test.shape[0], 8))
    model = SVC().fit(X_train, y_train) # C=1, kernel=rbf, gamma=auto
    predictions = model.predict(X_test)

    sample[sample.columns[t + 1]] = predictions

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/uti

KeyboardInterrupt: 

In [ ]:
#KMeans
from sklearn.cluster import KMeans

for t in range(28):
    X_train = pd.DataFrame(X_train_total[t])
    y_train = pd.DataFrame(df_light_y[df_light_y.columns[t]])
    X_test = pd.DataFrame(X_test_total[t])
    X_train = np.reshape(X_train, (X_train.shape[0], 8))
    X_test = np.reshape(X_test, (X_test.shape[0], 8))
    model = KMeans(n_clusters=2).fit(X_train, y_train)
    predictions = model.predict(X_test)

    sample[sample.columns[t + 1]] = predictions

In [ ]:
#KNN
from sklearn.neighbors import KNeighborsClassifier

for t in range(28):
    X_train = pd.DataFrame(X_train_total[t])
    y_train = pd.DataFrame(df_light_y[df_light_y.columns[t]])
    X_test = pd.DataFrame(X_test_total[t])
    X_train = np.reshape(X_train, (X_train.shape[0], 8))
    X_test = np.reshape(X_test, (X_test.shape[0], 8))
    model = KNeighborsClassifier().fit(X_train, y_train)
    predictions = model.predict(X_test)

    sample[sample.columns[t + 1]] = predictions

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:207: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:207: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:207: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/neighb

In [ ]:
sample

,user_id,time_slot_0,time_slot_1,time_slot_2,time_slot_3,time_slot_4,time_slot_5,time_slot_6,time_slot_7,time_slot_8,...,time_slot_18,time_slot_19,time_slot_20,time_slot_21,time_slot_22,time_slot_23,time_slot_24,time_slot_25,time_slot_26,time_slot_27
0,30460,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,30461,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30462,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,30463,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,30464,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7611,38071,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7612,38072,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
7613,38073,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7614,38074,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#輸出資料
h = sample.columns.tolist()
sample.to_csv('/Users/nuo/Documents/DA/kaggle/data-analytics-datagame/sample.csv', index=False, header=h)